#Importing necessary libraries

In [ ]:
!pip install transformers
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rouge-1.0.1-py3-none-any.whl (13 kB)


In [ ]:
import re
import string
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge import Rouge
import numpy as np
import sys

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Pre-Processing

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Mini Project/dataset.csv')

def remove_date(df):
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    df = df.applymap(lambda x: re.sub(pattern, '', str(x)))
    return df

urls = r'\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'

def remove_url(df):
  df = df.applymap(lambda x: re.sub(urls, '', str(x)))
  df = df.applymap(lambda x: re.sub('URL: ', '', str(x)))
  return df

foot = 'LIIofIndia: Copyright Policy | Disclaimers | Privacy Policy | Feedback'

def remove_foot(df):
  df = df.applymap(lambda x: re.sub(foot, '', str(x)))
  return df

def remove_punct(df):
    df = df.applymap(lambda x: x.translate(str.maketrans("", "", string.punctuation)))
    return df

data = remove_foot(data)
data = remove_date(data)
data = remove_url(data)
data = remove_punct(data)


#Defining Model

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#Generating Summaries

In [ ]:
summaries = []
for i in range(0, 10):

    input_text = data.iloc[i, 0]
    input_ids = tokenizer.encode(input_text,padding=True, truncation=True, return_tensors='pt').to(device)
    max_length = model.config.max_position_embeddings
    summary_ids = model.generate(input_ids, max_length=max_length, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)
    summaries.append(summary)

In [ ]:
sys.setrecursionlimit(10**6)

#Results

In [ ]:
rouge1_score = []
rouge2_score = []
rougeL_score = []

for i in range(0, 10):
    rouge = Rouge()
    scores = rouge.get_scores(summaries[i],data.iloc[i,1])
    print("Rouge 1 score of ", i," sample is - ", scores[0]["rouge-1"])
    print("Rouge 2 score of ", i," sample is - ", scores[0]["rouge-2"])
    print("Rouge l score of ", i," sample is - ", scores[0]["rouge-l"])
    rouge1_score.append(scores[0]["rouge-1"]["f"])
    rouge2_score.append(scores[0]["rouge-2"]["f"])
    rougeL_score.append(scores[0]["rouge-l"]["f"])



In [ ]:
print("Average rouge 1 score is", np.mean(rouge1_score))
print("Average rouge 2 score is", np.mean(rouge2_score))
print("Average rouge l score is", np.mean(rougeL_score))